In [1]:
%pylab inline
import cosima_cookbook as cc
import numpy as np
import matplotlib.pyplot as plt # to generate plots
import xarray as xr
import scipy as sp
import pandas as pd
import IPython.display
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap # plot on map projections
from glob import glob
import scipy.interpolate
import scipy.ndimage
from tqdm import tqdm_notebook
import pylab

# for georefferenced data
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm # for colormap
import netCDF4 as nc
import dask.array as da
import dataset

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Creating my own database:
db = '/g/data/e14/ss2778/cc_test1.db'
session = cc.database.create_session(db)

In [3]:
from joblib import Memory
memory = Memory(cachedir='/g/data/e14/ss2778/tmp/',verbose=0)

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/g/data/e14/ss2778/tmp/'", use "location='/g/data/e14/ss2778/tmp/'" instead.
  


In [4]:
dir_list=['/g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf9091']
cc.database.build_index(dir_list,session,update=True)

Indexing experiment: 01deg_jra55v13_ryf9091


0it [00:00, ?it/s]


0

In [ ]:
%time
expt =  '01deg_jra55v13_ryf9091'
var1 = 'uocn_m'
var2 = 'vocn_m'
uocn_m = cc.querying.getvar(expt, var1, session,start_time='1900-01-01',
                            end_time='1920-12-31' )
vocn_m = cc.querying.getvar(expt, var1, session,start_time='1900-01-01',
                            end_time='1920-12-31')

CPU times: user 0 ns, sys: 6 µs, total: 6 µs
Wall time: 8.82 µs


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

In [ ]:
@memory.cache
def calc_eke_dask(expt):

    u = da.from_array(uocn_m,chunks=(60,675,900)) 
    #u = da.concatenate(u_dataarrays, axis=0)

    v = da.from_array(vocn_m,chunks=(60,675,900)) 
    #v = da.concatenate(v_dataarrays, axis=0)

    u_mean = u.mean(axis=0)
    v_mean = v.mean(axis=0)

    u_ = u - u_mean
    v_ = v - v_mean

    EKE = 0.5 * (u_**2 + v_**2)
    EKE_mean = EKE.mean(axis=0)
    
  #  EKE = cc.compute_by_block(EKE)
    
    EKE=EKE.compute()
    return EKE

In [ ]:
%%time
EKE = calc_eke_dask('01deg_jra55v13_ryf9091')

In [ ]:
#Saving EKE field in .nc file:
new_1 = './EKE.nc'
print ('saving to ', new_1)
EKE.to_netcdf(path=new_1)
print ('finished saving')

In [ ]:
plt.figure(figsize=(15,8))
EKE_mean=EKE.mean('time')
X,Y=np.meshgrid(EKE.xu_ocean,EKE.yu_ocean)
h=plt.contourf(X,Y,EKE_mean,levels=100)
cb = plt.colorbar(h,orientation='vertical')
plt.contour(X,Y,EKE,levels=14, linewidths=0.5, colors='k')
plt.title('EKE (m2/s2)')
plt.ylabel('latitude(degrees)')
plt.xlabel('longitude(degrees)')
savefigure('EKE_mean')